# Expirementing data loading time with Pandas vs. Dask

File size for comparison:
- Small size file: 75MB
- Medium size file: 1.8GB
- Large size file: 12.7GB

Who is faster at what condition?

In [2]:
import pandas as pd
import numpy as np
import dask.dataframe
import multiprocessing as mp
import psutil
import string 
import random 
import time

In [3]:
time.time()

1585927178.1756659


# small data

## 00_Generating a CSV file with random data ~75 MB


In [4]:
# setting the number of rows for the CSV file (1M)
N = 1000000

# creating a pandas dataframe (df) with 7 columns with column names from A to G
df = pd.DataFrame(
    np.random.randint(9,999, size=(N, 7)), 
    columns=list('ABCDEFG'))

# creating one column 'H' of float type using the uniform distribution
df['H'] = np.random.rand(N)

# creating two additional columns with random strings
df['I'] = pd.util.testing.rands_array(10, N)
df['J'] = pd.util.testing.rands_array(10, N)

# expect: 1M rows x 10 columns
df.shape

(1000000, 10)

In [21]:
# export the dataframe to csv ~75MB
df.to_csv("test_data_75MB.csv", index=False)

## 01_pandas_read

In [24]:
start_time = time.time()

data = pd.read_csv("test_data_75MB.csv") 

print("%s seconds" % (time.time() - start_time))

1.2899370193481445 seconds


## 02_pandas_with_chunsize_option

In [25]:
start_time = time.time()

df_chunk = pd.read_csv("test_data_75MB.csv", chunksize=100000) # 10% of total rows#
chunk_list = []  

for chunk in df_chunk:  
    chunk_list.append(chunk)
data = pd.concat(chunk_list)

print("%s seconds" % (time.time() - start_time))

1.4829139709472656 seconds


## 03_dask_read

In [23]:
start_time = time.time()

data = dask.dataframe.read_csv("test_data_75MB.csv").compute()

print("%s seconds" % (time.time() - start_time))

1.3889448642730713 seconds


With small file loading, read_csv without chunsize is ideal. Adding chunsize option actually make it less efficient.

# medium data

## 00_Generating a CSV file with random data ~2 GB

In [25]:
# setting the number of rows for the CSV file (10M)
N = 10000000

# creating a pandas dataframe (df) with 8 columns and N rows with random integers between 999 and 999999 and with column names from A to H
df = pd.DataFrame(
    np.random.randint(9,999, size=(N, 7)), 
    columns=list('ABCDEFG'))

# creating one column 'H' of float type using the uniform distribution
df['H'] = np.random.rand(N)

# creating two additional columns with random strings
df['I'] = pd.util.testing.rands_array(10, N)
df['J'] = pd.util.testing.rands_array(10, N)
df['K'] = pd.util.testing.rands_array(10, N)
df['L'] = pd.util.testing.rands_array(10, N)
df['M'] = pd.util.testing.rands_array(10, N)
df['N'] = pd.util.testing.rands_array(10, N)
df['O'] = pd.util.testing.rands_array(10, N)

# expect: 10M rows x 15 columns
df.shape

(10000000, 15)

In [34]:
# export the dataframe to csv 1.8GB
df.to_csv("test_data_1GB.csv", index=False)

## 01_pandas_read

In [36]:
start_time = time.time()

df = pd.read_csv("test_data_1GB.csv") 

print("%s seconds" % (time.time() - start_time))

100.06393694877625 seconds


## 02_pandas_with_chunsize_option

In [28]:
start_time = time.time()

df_chunk = pd.read_csv("test_data_1GB.csv", chunksize=100000) 
chunk_list = []  

for chunk in df_chunk:  
    chunk_list.append(chunk)
df_concat = pd.concat(chunk_list)

print("%s seconds" % (time.time() - start_time))

66.94773411750793 seconds


## 03_dask_dataframe

In [29]:
start_time = time.time()

data = dask.dataframe.read_csv("test_data_1GB.csv").compute()

print("%s seconds" % (time.time() - start_time))

63.132309913635254 seconds


When file size gets to ~2GB, doing something more than `read_csv` shows significant benefit. Dask is slightly faster than pandas with chunksize, but similar performance.

Most functions used with pandas can be also use with dask. The differences arise from the parallel nature of dask.

Unlike pandas, with dask the data is not fully loaded into memory but is ready to be processed. Certain opperations can be performed again without loading the whole dataset into memory. 

# large data

## 00_Generating a CSV file with random data ~10 GB

In [39]:
# setting the number of rows for the CSV file (50M)
N = 50000000

# creating a pandas dataframe (df) with 10 columns and N rows with random between 99 and 9999 
df = pd.DataFrame(
    np.random.randint(99,9999, size=(N, 10)), 
    columns=list('ABCDEFGXYZ'))

# creating one column 'H' of float type using the uniform distribution
df['H1'] = np.random.rand(N)
df['H2'] = np.random.rand(N)
df['H3'] = np.random.rand(N)
df['H4'] = np.random.rand(N)
df['H5'] = np.random.rand(N)

# creating additional columns with random strings
df['K'] = pd.util.testing.rands_array(10, N)
df['L'] = pd.util.testing.rands_array(10, N)
df['M'] = pd.util.testing.rands_array(10, N)
df['N'] = pd.util.testing.rands_array(10, N)
df['O'] = pd.util.testing.rands_array(10, N)

df['P'] = pd.util.testing.rands_array(10, N)
df['Q'] = pd.util.testing.rands_array(10, N)
df['R'] = pd.util.testing.rands_array(10, N)
df['S'] = pd.util.testing.rands_array(10, N)
df['T'] = pd.util.testing.rands_array(10, N)

# expect: 50M rows x 25 columns
df.shape

(50000000, 25)

In [40]:
# export the dataframe to csv 12.7GB
df.to_csv("test_data_10GB.csv", index=False)

## 01_pandas_read

In [41]:
start_time = time.time()

data = dask.dataframe.read_csv("test_data_10GB.csv").compute()

print("%s seconds" % (time.time() - start_time))

572.7035613059998 seconds


## 02_pandas_with_chunsize_option

In [4]:
start_time = time.time()

df_chunk = pd.read_csv("test_data_10GB.csv", chunksize=100000) 
chunk_list = []  

for chunk in df_chunk:  
    chunk_list.append(chunk)
data = pd.concat(chunk_list)

print("%s seconds" % (time.time() - start_time))

471.1917221546173 seconds


## 03_dask_dataframe

In [5]:
start_time = time.time()

data = dask.dataframe.read_csv("test_data_10GB.csv").compute()

print("%s seconds" % (time.time() - start_time))

787.500736951828 seconds


# Chunsize
Regarding chunksize:
https://stackoverflow.com/questions/35235010/what-is-the-optimal-chunksize-in-pandas-read-csv-to-maximize-speed

There is no "optimal chunksize". Because chunksize only tells you the number of rows per chunk, not the memory-size of a single row, hence it's meaningless to try to make a rule-of-thumb on that. ([*] although generally I've only ever seen chunksizes in the range 100..64K)

To get memory size, you'd have to convert that to a memory-size-per-chunk or -per-row...

by looking at your number of columns, their dtypes, and the size of each; use either df.describe(), or else for more in-depth memory usage, by column.

In [3]:
# Check the number of cores and memory usage
cpu = psutil.cpu_count()
memory = psutil.virtual_memory().total
memory/cpu

2147483648.0

### Experiment:
- 1k
- 10k
- 100k
- 1m
- 5m

## Small Data: 75 MB

In [2]:
# 1K
start_time = time.time()

df_chunk = pd.read_csv("test_data_75MB.csv", chunksize=1000) 
chunk_list = []  

for chunk in df_chunk:  
    chunk_list.append(chunk)
data = pd.concat(chunk_list)

print("%s seconds" % (time.time() - start_time))

4.595816135406494 seconds


In [3]:
# 10K
start_time = time.time()

df_chunk = pd.read_csv("test_data_75MB.csv", chunksize=10000) 
chunk_list = []  

for chunk in df_chunk:  
    chunk_list.append(chunk)
data = pd.concat(chunk_list)

print("%s seconds" % (time.time() - start_time))

1.9769558906555176 seconds


In [4]:
# 100K
start_time = time.time()

df_chunk = pd.read_csv("test_data_75MB.csv", chunksize=100000) 
chunk_list = []  

for chunk in df_chunk:  
    chunk_list.append(chunk)
data = pd.concat(chunk_list)

print("%s seconds" % (time.time() - start_time))

1.4228427410125732 seconds


In [5]:
# 1M
start_time = time.time()

df_chunk = pd.read_csv("test_data_75MB.csv", chunksize=1000000) 
chunk_list = []  

for chunk in df_chunk:  
    chunk_list.append(chunk)
data = pd.concat(chunk_list)

print("%s seconds" % (time.time() - start_time))

1.4417939186096191 seconds


## Medium data: 1.8 GB

In [10]:
# 1k
start_time = time.time()

df_chunk = pd.read_csv("test_data_1GB.csv", chunksize=1000) 
chunk_list = []  

for chunk in df_chunk:  
    chunk_list.append(chunk)
df_concat = pd.concat(chunk_list)

print("%s seconds" % (time.time() - start_time))

144.5809988975525 seconds


In [12]:
# 10k
start_time = time.time()

df_chunk = pd.read_csv("test_data_1GB.csv", chunksize=10000) 
chunk_list = []  

for chunk in df_chunk:  
    chunk_list.append(chunk)
df_concat = pd.concat(chunk_list)

print("%s seconds" % (time.time() - start_time))

106.53525710105896 seconds


In [13]:
# 100k
start_time = time.time()

df_chunk = pd.read_csv("test_data_1GB.csv", chunksize=100000) 
chunk_list = []  

for chunk in df_chunk:  
    chunk_list.append(chunk)
df_concat = pd.concat(chunk_list)

print("%s seconds" % (time.time() - start_time))

100.17744207382202 seconds


In [14]:
# 1M
start_time = time.time()

df_chunk = pd.read_csv("test_data_1GB.csv", chunksize=1000000) 
chunk_list = []  

for chunk in df_chunk:  
    chunk_list.append(chunk)
df_concat = pd.concat(chunk_list)

print("%s seconds" % (time.time() - start_time))

108.55591487884521 seconds


In [15]:
# 1M
start_time = time.time()

df_chunk = pd.read_csv("test_data_1GB.csv", chunksize=5000000) 
chunk_list = []  

for chunk in df_chunk:  
    chunk_list.append(chunk)
df_concat = pd.concat(chunk_list)

print("%s seconds" % (time.time() - start_time))

115.60315918922424 seconds


Take away:
Experiment 10k~1M and you'll likely find a optimal scale by trying within this range.

## Summary

**75 MB, 1M rows**
- 1k chunsize -> 4.59s
- 10k chunsize -> 1.97s
- **100k chunsize -> 1.42s** `winner`
- 1M chunsize -> 1.44s

**1.8 GB, 10M rows**
- 1k chunsize -> 144.58s
- 10k chunsize -> 106.53s
- **100k chunsize -> 100.17s** `winner`
- 1M chunsize -> 115.60s

Take-away:
- There's an optimal level of chunsize we can pick based on the size of each row of our dataframe;
- Picking a larger than optimal chunsize is more effiicent than picking a smaller one;